# Constructing and attacking NTRU public key cryptosystem
In contrast to cryptosystems such as RSA, Diffie-Hellman or ECC which are based on Group operations, the NTRU cryptosystem is a ring-based cryptosystem, specifically convolution polynomial rings, but it's underlying hard mathematical problem can also be interpreted as **SVP** or **CVP** in a lattice.

# NTRUEncrypt
The NTRU public key cryptosystem, parameterized by integers $N$, $p$, $q$ and $d$ such that
$$
\begin{aligned}
\text{1.}&\; N \in \mathbb{P} \; \text{is prime} \newline
\text{2.}&\; \gcd(N,q) = \gcd(p,q) = 1 \newline
\text{3.}&\; d > 0 \newline
\text{4.}&\; q > (6d + 1)p
\end{aligned}
$$
is based on operations in three **polynomial rings** $R, R_p, R_q$
$$
R = \frac{\mathbb{Z}[X]}{X^{N} - 1}, \quad R_p = \frac{\mathbb{Z}_p[X]}{X^{N} - 1}, \quad R_q = \frac{\mathbb{Z}_q[X]}{X^{N} - 1}
$$
and **ternary polynomials** defined for any positive integers $d_1$ and $d_2$ as
$$
\mathcal{T}(d_1, d_2) = \left\{ a(x) \in R \; \middle| \;
\begin{array}{l}
a(x) \; \text{has} \; d_1 \; \text{coefficients equal to 1,}\newline
a(x) \; \text{has} \; d_2 \; \text{coefficients equal to -1,}\newline
a(x) \; \text{has all other coefficients equal to 0}
\end{array}
\right\}
$$

# Key creation 
Let $(N, p, q, d)$ be publicly known parameters of the NTRU cryptosystem, chosen by some trusted authority.  
**Private  key** consists of two randomly chosen polynomials
$$
f(x) \in \mathcal{T}(d + 1, d) \qquad \text{and} \qquad g(x) \in \mathcal{T}(d, d)
$$
To set up public key we first need to calculate inverses
$$
F_q(x) = f(x)^{-1} \; \text{in} \; R_q \qquad \text{and} \qquad F_p = f(x)^{-1} \; \text{in} \; R_p
$$
**The $f(x)$ must be chosen such that inverses $F_q(x)$ and $F_p(x)$ exists**.

The polynomial $h(x)$ defined as
$$
h(x) = F_q(x) \cdot g(x) \; \text{in} \; R_q
$$
will be a **public key**

Private key: $(f(x), g(x))$  
Public key: $h(x)$

In [1]:
import numpy as np
import lbpqc.primitives.polynomial.NTRU_polynomial_ring as npr
import lbpqc.primitives.polynomial.modulo_integer_polynomial_ring as mpr
# Let's choose NTRU parameters and private key (f,g)
N, p, q, d = 7, 3, 41, 2

f = np.array([-1, 0, 1, 1, -1, 0, 1], dtype=int) #f(x) = x^6 - x^4 + x^3 + x^2 - 1
g = np.array([0, -1, -1, 0, 1, 0, 1], dtype=int) #g(x) = x^6 + x^4 - x^2 - x

# Now lets compute inverses Fq and Fp
Fq = npr.ntru_poly_inverse(N, q, f)
Fp = npr.ntru_poly_inverse(N, p, f)

# Lets check if the inverses are correct
print(npr.ntru_poly_mul(N, q, Fq, f))
print(npr.ntru_poly_mul(N, p, Fp, f))

# Now let's calculate public key h
h = npr.ntru_poly_mul(N, q, Fq, g)
print(h)

print("Private key: (f(x), h(x))")
print("f(x) = ", f)
print("g(x) = ", g)
print()
print("Public key: h(x)")
print("h(x) = ", h)

[1 0 0 0 0 0 0]
[1 0 0 0 0 0 0]
[30 26  8 38  2 40 20]
Private key: (f(x), h(x))
f(x) =  [-1  0  1  1 -1  0  1]
g(x) =  [ 0 -1 -1  0  1  0  1]

Public key: h(x)
h(x) =  [30 26  8 38  2 40 20]


# Encryption
Plaintext $m$ has a form of polynomial $m(x) \in R$ whose coefficients satisfy $ \; -\frac{1}{2}p < m_i \leq \frac{1}{2}p $.  
In order to encrypt $m$ we also need a random polynomial $r(x) \in \mathcal{T}(d,d)$ then
$$
e(x) \equiv p h(x) \cdot r(x) + m(x) \quad \mod q
$$
is a **ciphertext** and $e(x) \in R_q$

In [2]:
# plaintext
m = np.array([ 1,-1, 1, 1, 0,-1], dtype=int)
print("plaintext m(x) =", m)
# random element
r = np.array([-1, 1, 0, 0, 0,-1, 1], dtype=int)
print("random element r(x) =", r) 

print()
# ciphertext
e = mpr.poly_mod_add(npr.ntru_poly_mul(N, q, mpr.poly_mod_scalar_mul(h, p, modulus=q), r), m, q)
print("ciphertext e(x) =", e)

plaintext m(x) = [ 1 -1  1  1  0 -1]
random element r(x) = [-1  1  0  0  0 -1  1]

ciphertext e(x) = [25  3 40  2  4 19 31]


# Decryption
To decrypt ciphertext $e(x)$ we first need to compute
$$
a(x) \equiv f(x) \cdot e(x) \quad \mod q
$$
then
$$
b(x) \equiv F_p(x) \cdot \text{center\_lift} (a(x)) \quad \mod p
$$
is equal to the plaintext $m(x)$

In [3]:
a = npr.ntru_poly_mul(N, q, f, e)
a = mpr.center_lift(a, q)

b = npr.ntru_poly_mul(N, p, Fp, a)
b = mpr.center_lift(b, p)

print(f"Message m: {m}")
print(f"Decrypted message b: {b}")

Message m: [ 1 -1  1  1  0 -1]
Decrypted message b: [ 1 -1  1  1  0 -1  0]


# NTRU as a lattice
Given NTRU cryptosystem with parameters $(N,p,q,d)$ we are going to identify each pair of polynomials
$$
a(x) = a_0 + \ldots + a_{N-1}x^{N-1} \qquad \text{and} \qquad b(x) = b_0 + \ldots + b_{N-1}x^{N-1}
$$
in $R$ with $2N$-dimensional vector
$$
(a,b) = (a_0, \ldots, a_{N-1}, b_0, \ldots, b_{N-1}) \in \mathbb{Z}^{2N}
$$

In [4]:
from lbpqc.primitives.lattice import full_rank_lattice as flr
from lbpqc.primitives.lattice import reduction

M_NTRU = flr.NTRU_lattice_basis((N,p,q,d), h)
print(M_NTRU.astype(int))

[[ 1  0  0  0  0  0  0 30 26  8 38  2 40 20]
 [ 0  1  0  0  0  0  0 20 30 26  8 38  2 40]
 [ 0  0  1  0  0  0  0 40 20 30 26  8 38  2]
 [ 0  0  0  1  0  0  0  2 40 20 30 26  8 38]
 [ 0  0  0  0  1  0  0 38  2 40 20 30 26  8]
 [ 0  0  0  0  0  1  0  8 38  2 40 20 30 26]
 [ 0  0  0  0  0  0  1 26  8 38  2 40 20 30]
 [ 0  0  0  0  0  0  0 41  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 41  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 41  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 41  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 41  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 41  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 41]]


In [5]:
M_NTRU_LLL = reduction.LLL(M_NTRU)
sv = M_NTRU_LLL[0].astype(int)
print(sv)
f_prim = sv[:N]
g_prim = sv[N:]
print(f_prim)
print(g_prim)

Fprim_q = npr.ntru_poly_inverse(N, q, f_prim)
Fprim_p = npr.ntru_poly_inverse(N, p, f_prim)
print(Fprim_q)
print(Fprim_p)

[ 1  0 -1  1  0 -1 -1 -1  0 -1  0  1  1  0]
[ 1  0 -1  1  0 -1 -1]
[-1  0 -1  0  1  1  0]
[20 10 15 33  4 39  1]
[2 0 1 2 2 2 2]


In [6]:
aprim = npr.ntru_poly_mul(N, q, f_prim, e)
aprim = mpr.center_lift(aprim, q)

bprim = npr.ntru_poly_mul(N, p, Fprim_p, aprim)
bprim = mpr.center_lift(bprim, p)

print(f"Message m: {m}")
print(f"Decrypted message b: {b}")

Message m: [ 1 -1  1  1  0 -1]
Decrypted message b: [ 1 -1  1  1  0 -1  0]
